In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
test_df=pd.read_csv("testData.tsv",delimiter='\t')
train_df=pd.read_csv("labeledTrainData.tsv",delimiter='\t')

In [ ]:
train_df.drop("id",axis=1,inplace=True)
test_df.drop('id',axis=1,inplace=True)

In [ ]:
from IPython.display import display,Markdown

In [ ]:
 train_df.sample(4)

In [ ]:
corpus=train_df.review.values

In [ ]:
import gensim 
from gensim.utils import simple_preprocess 

In [ ]:
dct=gensim.corpora.Dictionary()

In [ ]:
extra={"not", "good", "but", "great", "bad",  "well" , "best", "love", "plot", "better",'right','without','few','funny','horror',}

In [ ]:
toksTrain=[simple_preprocess(doc) for doc in train_df.review]
toksTest=[simple_preprocess(doc) for doc in test_df.review]

In [ ]:
import pickle

In [ ]:
def lemmatized(toks):
    file=open(toks,'rb')
    return pickle.load(file)
#     return [[i.lemma_ for i in nlp(" ".join(hlf))] for hlf in toks]

In [ ]:
toksTrainLemmatized=lemmatized("../input/pklling/toksTrainLemmatized.pkl")
toksTestLemmatized=lemmatized('../input/pklling/toksTestLemmatized.pkl')

In [ ]:
# toksTestLemmatized=lemmatized(toksTest)

In [ ]:
# import pickle 
# pickle.dump(toksTrainLemmatized,open("toksTrainLemmatized.pkl",'wb'))
# pickle.dump(toksTestLemmatized,open("toksTestLemmatized.pkl",'wb'))

In [ ]:
dct=gensim.corpora.Dictionary(toksTrainLemmatized)
dct.filter_n_most_frequent(180)

In [ ]:
len(dct)

In [ ]:
validKeys=set(sorted(dct.keys(),key=lambda k:-dct.cfs[k])[:7000])

In [ ]:
dct.filter_tokens(good_ids=validKeys)

In [ ]:
dct.add_documents([extra])

In [ ]:
trainCorpus=[dct.doc2bow(doc) for doc in toksTrainLemmatized]
testCorpus=[dct.doc2bow(doc) for doc in toksTestLemmatized]

In [ ]:
from gensim.models import TfidfModel

In [ ]:
tfidf=TfidfModel(trainCorpus,smartirs='ntc')

In [ ]:
trainIdf=tfidf[trainCorpus]
testIdf=tfidf[testCorpus]

In [ ]:
trainIdfDict=[dict(idf) for idf in trainIdf]

In [ ]:
testIdfDict=[dict(idf) for idf in testIdf]

In [ ]:
train_X=pd.DataFrame(trainIdfDict)
test_X=pd.DataFrame(testIdfDict,columns=train_X.columns)

In [ ]:
assert all([i in train_X.columns for i in test_X])

In [ ]:
assert all(train_X.columns==test_X.columns)

In [ ]:
train_y=train_df.sentiment.values

In [ ]:
len(train_X.columns),len(test_X.columns)

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
train_X.fillna(0,inplace=True)
test_X.fillna(0,inplace=True)

In [ ]:
train_X.columns==test_X.columns

In [ ]:
train_vX,test_vX,train_vy,test_vy=tts(train_X,train_y,test_size=0.2,random_state=42)

In [ ]:
clf=LinearSVC(C=.18,verbose=1)

clf.fit(train_X,train_y)
clf.score(test_vX,test_vy)

In [ ]:
preds=clf.predict(test_X)

In [ ]:
submit=pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv")

In [ ]:
submit.sentiment=preds

In [ ]:
tst_df=pd.read_csv("testData.tsv",delimiter='\t')

In [ ]:
all(tst_df.index==submit.index)

In [ ]:
submit.to_csv("submission.csv",index=False)